In [5]:
import corpus as corpus_class
import categories, filters, vocabulary_builders
from feature_extractors import multinomial_model, tfidf
from filters import std_filters

import numpy as np

In [6]:
qfile_train = 'question_train.csv'
qcatfile_train = 'question_category_train.csv'
catfile = 'category.csv'
qfile_test = 'question_test.csv'

In [7]:
filtees = std_filters()
filtees2 = filtees.copy()
filtees2['word_filters'] = [filters.spell_corrector] + filtees2['word_filters']
filtees3 = filtees.copy()
filtees3['word_filters'] = []

In [8]:
corpus = corpus_class.corpus( categories.categories() )
corpus.load(qfile_train, qcatfile_train)
corpus.process(**filtees, corpus_size=-1)

In [9]:
corpus.spell_checker.vocabulary

Counter({'rupert': 8,
         '1970': 2,
         'schwul': 104,
         'schorah': 2,
         'apps': 4,
         'cobain': 12,
         'muenzen': 2,
         'luxor': 2,
         'katsuie': 2,
         'cd': 52,
         'glatt': 4,
         'dziwnow': 4,
         'gundus': 2,
         'jähriger': 36,
         'meinen': 452,
         '3:0': 2,
         'gmbh': 6,
         'pantherin': 4,
         'eckardt': 2,
         'kontinent': 12,
         'oberpfalz': 2,
         'explorer': 2,
         'filmstars': 2,
         'zuständig': 2,
         'contra': 2,
         'ausprobiert': 2,
         'kalender': 2,
         'privatleben': 2,
         'ablaufhügel': 2,
         'franka': 32,
         'jugend': 80,
         'hans+iris': 2,
         'bedenklich': 2,
         'zuzustellen': 2,
         'intoleranz': 2,
         'streichholz': 2,
         'koks': 4,
         'trek': 16,
         'kränken': 2,
         'lottenzahlen': 2,
         'borderline': 6,
         'hochhäuser': 2,
       

In [10]:
corpus.simple_split(0.3)

In [11]:
corpus.make_features(-1)

In [12]:
print(corpus)

TypeError: Can't convert 'bool' object to str implicitly

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(corpus.X_tr, corpus.y_tr)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [14]:
clf.score(corpus.X_te, corpus.y_te)

0.5977808599167822

In [15]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=2.0)
clf.fit(corpus.X_tr, corpus.y_tr)
clf.score(corpus.X_te, corpus.y_te)

0.59153952843273228

In [16]:
corpus.X_tr

<10091x11469 sparse matrix of type '<class 'numpy.float64'>'
	with 43386 stored elements in Compressed Sparse Row format>